In [15]:
        import pandas as pd
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.model_selection import train_test_split
        from sklearn.neighbors import KNeighborsClassifier
        from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
        import re
        import nltk
        from nltk.sentiment.vader import SentimentIntensityAnalyzer
        from gensim import corpora
        import gensim
        df = pd.read_csv('spam.csv', encoding='latin-1')
        nltk.download('vader_lexicon')
        sia = SentimentIntensityAnalyzer()
        df['sentiment_score'] = df['v2'].apply(lambda x: sia.polarity_scores(x)['compound'])
        texts = [[word for word in document.lower().split()] for document in df['v2']]
        dictionary = corpora.Dictionary(texts)
        corpus = [dictionary.doc2bow(text) for text in texts]
        lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=15)
        df['topic_model'] = [lda_model.get_document_topics(doc) for doc in corpus]
        df['v2'] = df['v2'].apply(lambda x: x.lower())
        df['v2'] = df['v2'].apply(lambda x: re.sub(r'[^\\w\\s]', '', x))

        X = df[['v2', 'sentiment_score', 'topic_model']]
        y = df['v1']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        vectorizer = TfidfVectorizer(max_features=5000)
        X_train_tfidf = vectorizer.fit_transform(X_train['v2'])
        X_test_tfidf = vectorizer.transform(X_test['v2'])

        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(X_train_tfidf, y_train)
        y_pred = knn.predict(X_test_tfidf)

        print('Accuracy:', accuracy_score(y_test, y_pred))
        print('Classification Report:')
        print(classification_report(y_test, y_pred))
        print('Confusion Matrix:')
        print(confusion_matrix(y_test, y_pred))











[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Accuracy: 0.8825112107623319
Classification Report:
              precision    recall  f1-score   support

         ham       0.89      0.98      0.94       965
        spam       0.69      0.23      0.34       150

    accuracy                           0.88      1115
   macro avg       0.79      0.61      0.64      1115
weighted avg       0.86      0.88      0.86      1115

Confusion Matrix:
[[950  15]
 [116  34]]


In [18]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
df = pd.read_csv('spam.csv', encoding='latin-1')
import spacy
nlp = spacy.load('en_core_web_sm')

df['named_entities'] = df['v2'].apply(lambda x: [ent.text for ent in nlp(x).ents])

from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

df['pos_tags'] = df['v2'].apply(lambda x: [pos for word, pos in pos_tag(word_tokenize(x))])

df['v2'] = df['v2'].apply(lambda x: x.lower())
df['v2'] = df['v2'].apply(lambda x: re.sub(r'[^\\w\\s]', '', x))

X = df[['v2', 'named_entities', 'pos_tags']]
y = df['v1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train['v2'])
X_test_tfidf = vectorizer.transform(X_test['v2'])
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train_tfidf, y_train)

y_pred = rfc.predict(X_test_tfidf)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Accuracy: 0.8941704035874439
Classification Report:
              precision    recall  f1-score   support

         ham       0.91      0.98      0.94       965
        spam       0.70      0.37      0.49       150

    accuracy                           0.89      1115
   macro avg       0.80      0.67      0.71      1115
weighted avg       0.88      0.89      0.88      1115



In [21]:
from gensim.matutils import corpus2dense

nmf_model = gensim.models.Nmf(corpus=corpus, num_topics=5, passes=15)
df['topic_model_nmf'] = [nmf_model.get_document_topics(doc) for doc in corpus]



In [29]:
rfc = RandomForestClassifier(n_estimators=100)
X_train = df[['v2', 'named_entities', 'pos_tags']].iloc[X_train.index]
X_test = df[['v2', 'named_entities', 'pos_tags']].iloc[X_test.index]

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

for col in ['named_entities', 'pos_tags']:
    X_train[col] = X_train[col].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
    X_test[col] = X_test[col].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
print('Random Forest Accuracy:', accuracy_score(y_test, y_pred_rfc))
print('Random Forest Classification Report:')
print(classification_report(y_test, y_pred_rfc))
print('Random Forest Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_rfc))

Random Forest Accuracy: 0.9067264573991032
Random Forest Classification Report:
              precision    recall  f1-score   support

         ham       0.90      1.00      0.95       965
        spam       0.94      0.33      0.49       150

    accuracy                           0.91      1115
   macro avg       0.92      0.66      0.72      1115
weighted avg       0.91      0.91      0.89      1115

Random Forest Confusion Matrix:
[[962   3]
 [101  49]]


In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim import corpora
import gensim
import spacy
from textblob import TextBlob
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
df = pd.read_csv('spam.csv', encoding='latin-1')

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
df['sentiment_score'] = df['v2'].apply(lambda x: sia.polarity_scores(x)['compound'])
texts = [[word for word in document.lower().split()] for document in df['v2']]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=15)
df['topic_model'] = [lda_model.get_document_topics(doc) for doc in corpus]
nlp = spacy.load('en_core_web_sm')
df['named_entities'] = df['v2'].apply(lambda x: [ent.text for ent in nlp(x).ents])

df['pos_tags'] = df['v2'].apply(lambda x: [pos for word, pos in nltk.pos_tag(nltk.word_tokenize(x))])

df['sentiment_score_tb'] = df['v2'].apply(lambda x: TextBlob(x).sentiment.polarity)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [33]:
        import pandas as pd
        import tensorflow as tf
        from tensorflow.keras.layers import Embedding, Dense, Dropout, Flatten
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.preprocessing.text import Tokenizer
        from tensorflow.keras.preprocessing.sequence import pad_sequences
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import LabelEncoder


        df = pd.read_csv('spam.csv', encoding='latin-1')

        tokenizer = Tokenizer(num_words=5000)
        tokenizer.fit_on_texts(df['v2'])

        sequences = tokenizer.texts_to_sequences(df['v2'])
        padded = pad_sequences(sequences, maxlen=200)

        le = LabelEncoder()
        df['v1'] = le.fit_transform(df['v1'])

        X = padded
        y = df['v1']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = Sequential()
        model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

        loss, accuracy = model.evaluate(X_test, y_test)
        print('Accuracy:', accuracy)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.8999 - loss: 0.2863 - val_accuracy: 0.9749 - val_loss: 0.0847
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - accuracy: 0.9837 - loss: 0.0608 - val_accuracy: 0.9874 - val_loss: 0.0536
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.9941 - loss: 0.0358 - val_accuracy: 0.9812 - val_loss: 0.0863
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.9971 - loss: 0.0245 - val_accuracy: 0.9892 - val_loss: 0.0560
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.9991 - loss: 0.0202 - val_accuracy: 0.9857 - val_loss: 0.0546
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9884 - loss: 0.0398
Accuracy: 0.9856502413749695


In [34]:
        import pandas as pd
        from sklearn.neighbors import KNeighborsClassifier
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.model_selection import train_test_split, GridSearchCV
        from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
        import re

        file_path = 'spam.csv'

        df = pd.read_csv(file_path, encoding='latin-1')

        df['v2'] = df['v2'].apply(lambda x: x.lower())
        df['v2'] = df['v2'].apply(lambda x: re.sub(r'[^\\w\\s]', '', x))

        X = df['v2']
        y = df['v1']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        vectorizer = TfidfVectorizer(max_features=5000)
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')
        knn.fit(X_train_tfidf, y_train)

        y_pred = knn.predict(X_test_tfidf)

        print('Accuracy:', accuracy_score(y_test, y_pred))
        print('Classification Report:')
        print(classification_report(y_test, y_pred))
        print('Confusion Matrix:')
        print(confusion_matrix(y_test, y_pred))

        param_grid = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
        grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
        grid_search.fit(X_train_tfidf, y_train)

        print('Best Parameters:', grid_search.best_params_)
        print('Best Score:', grid_search.best_score_)

        best_knn = KNeighborsClassifier(**grid_search.best_params_)
        best_knn.fit(X_train_tfidf, y_train)

        y_pred_best = best_knn.predict(X_test_tfidf)

        print('Accuracy:', accuracy_score(y_test, y_pred_best))
        print('Classification Report:')
        print(classification_report(y_test, y_pred_best))
        print('Confusion Matrix:')
        print(confusion_matrix(y_test, y_pred_best))


Accuracy: 0.8825112107623319
Classification Report:
              precision    recall  f1-score   support

         ham       0.89      0.98      0.94       965
        spam       0.69      0.23      0.34       150

    accuracy                           0.88      1115
   macro avg       0.79      0.61      0.64      1115
weighted avg       0.86      0.88      0.86      1115

Confusion Matrix:
[[950  15]
 [116  34]]
Best Parameters: {'n_neighbors': 9, 'weights': 'distance'}
Best Score: 0.8786182703970447
Accuracy: 0.8977578475336323
Classification Report:
              precision    recall  f1-score   support

         ham       0.91      0.98      0.94       965
        spam       0.75      0.36      0.49       150

    accuracy                           0.90      1115
   macro avg       0.83      0.67      0.71      1115
weighted avg       0.89      0.90      0.88      1115

Confusion Matrix:
[[947  18]
 [ 96  54]]
